1. Importamos las librerías necesarias

In [ ]:
import fitz 
import docx
import re
import os
import nltk
import textstat
import numpy as np
import tkinter as tk
import google.generativeai as genai
from dotenv import load_dotenv
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tkinter import filedialog
from collections import Counter
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aoroz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aoroz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Se carga la API de Gemini, en donde dice API_KEY colocar la API de Gemini. Se obtiene en el siguiente link
https://aistudio.google.com/apikey

In [5]:
# Cargar la API key desde .env
load_dotenv()
genai.configure(api_key=os.getenv("API_KEY"))

Ya que al final crearemos un orquestador para todo el procedimiento, empezamos con las funciones.

La primera es para pedirle al usuario el temario de la asignatura a la que se le va generar el material

In [26]:
def request_file():
    while True:
        print("\nPlease enter the path to a PDF, DOCX, or TXT file:")
        file_path = input().strip()
        
        # Check if file exists
        if not os.path.exists(file_path):
            print("Error: File does not exist.")
            continue
            
        # Get file extension
        _, extension = os.path.splitext(file_path)
        extension = extension.lower()
        
        # Validate file extension
        valid_extensions = ['.pdf', '.docx', '.txt']
        if extension not in valid_extensions:
            print(f"Error: File must be one of these types: {', '.join(valid_extensions)}")
            continue
            
        return file_path

Luego, se realiza la extracción del contenido según el tipo de archivo que se haya cargado, en este caso puede ser PDF, DOCX o TXT

In [27]:
def extract_text_from_pdf(pdf_path):
    """Extrae texto de un archivo PDF."""
    with fitz.open(pdf_path) as doc:
        return "\n".join([page.get_text("text") for page in doc])

def extract_text_from_docx(docx_path):
    """Extrae texto de un archivo DOCX."""
    doc = docx.Document(docx_path)
    return "\n".join([para.text for para in doc.paragraphs])

def extract_text_from_txt(txt_path):
    """Lee texto desde un archivo TXT."""
    with open(txt_path, "r", encoding="utf-8") as file:
        return file.read()

def extract_text(file_path):
    """Detecta el tipo de archivo y extrae su contenido."""
    _, ext = os.path.splitext(file_path)
    if ext.lower() == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext.lower() == ".docx":
        return extract_text_from_docx(file_path)
    elif ext.lower() == ".txt":
        return extract_text_from_txt(file_path)
    else:
        raise ValueError("Formato de archivo no soportado. Usa PDF, DOCX o TXT.")


Se realiza la petición al modelo de Gemini para realizar el material de estudio mediante 5 prompts, los cuales le piden:
- Notas de clase
- Problemas
- Preguntas de discusión
- Objetivos de aprendizaje
- Lecturas sugeridas

In [28]:
def generate_content(prompt):
    """Genera contenido con Gemini a partir de un prompt."""
    model = genai.GenerativeModel("gemini-2.0-flash-001")
    response = model.generate_content(prompt)
    return response.text

def generate_study_materials(course_text):
    """Genera notas de clase, problemas, preguntas, objetivos y lecturas con Gemini."""
    
    prompts = {
        "notas_clase": f"Genera notas de clase detalladas basadas en este programa de curso:\n{course_text}",
        "problemas_practica": f"Genera problemas de práctica con soluciones basados en este programa de curso:\n{course_text}",
        "preguntas_discusion": f"Genera preguntas para discusión relacionadas con los temas de este programa de curso:\n{course_text}",
        "objetivos_aprendizaje": f"Genera objetivos de aprendizaje detallados basados en este programa de curso:\n{course_text}",
        "lecturas_sugeridas": f"Sugiere lecturas y recursos académicos basados en este programa de curso:\n{course_text}"
    }

    materials = {key: generate_content(prompt) for key, prompt in prompts.items()}
    
    return materials


Generamos el PDF que contiene la respuesta del LLM, a este PDF se le organizan cosas como las negritas, los bloques de código, entre otros elementos.

In [ ]:
# Estilo para bloques de código
code_style = ParagraphStyle(
    "CodeStyle",
    fontName="Courier",
    fontSize=10,
    leading=12,
    spaceBefore=5,
    spaceAfter=5,
    backColor="#F5F5F5"  # Fondo gris claro para destacar bloques de código
)

def format_text(text):
    """Convierte **negrita** en <b>negrita</b>."""
    text = re.sub(r'\*\*(.*?)\*\*', r'<b>\1</b>', text)  # Negritas
    return text

def process_sections(text):
    """Detecta código dentro de ``` y lo separa del texto normal."""
    sections = re.split(r"```", text)
    formatted_content = []
    
    for i, section in enumerate(sections):
        if i % 2 == 0:
            formatted_content.append(("text", format_text(section.strip())))  # Texto normal
        else:
            # Formatear código correctamente con saltos de línea
            formatted_code = section.strip().replace("\n", "<br/>")  # Saltos de línea
            formatted_code = formatted_code.replace(" ", "&nbsp;")   # Mantener espacios
            formatted_content.append(("code", formatted_code))
    
    return formatted_content

def generate_pdf(materials, output_filename):
    """Genera un PDF con materiales didácticos formateados correctamente."""
    output_filename = "../"+output_filename+".pdf"
    doc = SimpleDocTemplate(output_filename, pagesize=letter)
    styles = getSampleStyleSheet()
    content = []

    # Agregar título principal
    title = Paragraph("<b>Material Didáctico Generado</b>", styles["Title"])
    content.append(title)
    content.append(Spacer(1, 0.2 * inch))

    for section, text in materials.items():
        # Agregar título de sección
        section_title = Paragraph(f"<b>{section.replace('_', ' ').title()}</b>", styles["Heading2"])
        content.append(section_title)
        content.append(Spacer(1, 0.15 * inch))

        # Procesar bloques de texto y código
        formatted_sections = process_sections(text)
        for section_type, section_content in formatted_sections:
            if section_type == "text":
                paragraphs = section_content.split("\n\n")  # Separar párrafos
                for paragraph in paragraphs:
                    content.append(Paragraph(paragraph, styles["BodyText"]))
                    content.append(Spacer(1, 0.1 * inch))
            elif section_type == "code":
                # Formato de código con Courier y saltos de línea
                content.append(Paragraph(f'<font face="Courier">{section_content}</font>', code_style))
                content.append(Spacer(1, 0.2 * inch))

    # Generar PDF
    doc.build(content)
    print(f"✅ PDF generado correctamente: {output_filename}")



In [10]:
# Cargar stopwords en español
stopwords_es = stopwords.words('spanish')

# Cargar modelo de embeddings para similitud semántica
model = SentenceTransformer('all-MiniLM-L6-v2')

def concatenar_material(material):
    """
    Convierte un diccionario con materiales de clase en un solo string.
    """
    if isinstance(material, dict):
        return " ".join(str(v) for v in material.values())
    return str(material)

def calcular_relevancia(temario, material):
    """
    Calcula la similitud coseno entre el temario y el material generado usando TF-IDF.
    """
    vectorizer = TfidfVectorizer(stop_words=stopwords_es)
    tfidf_matrix = vectorizer.fit_transform([temario, material])
    similitud = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
    return similitud

def calcular_consistencia(seccion1, seccion2):
    """
    Mide la similitud semántica entre dos secciones del material.
    """
    emb1 = model.encode(seccion1, convert_to_tensor=True)
    emb2 = model.encode(seccion2, convert_to_tensor=True)
    similitud = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
    return similitud

def calcular_legibilidad(texto):
    """
    Calcula la puntuación Flesch-Kincaid para evaluar la legibilidad del texto.
    """
    return textstat.flesch_kincaid_grade(texto)

def analizar_terminologia(temario, material):
    """
    Evalúa la presencia de términos clave del temario en el material generado.
    """
    palabras_temario = set(nltk.word_tokenize(temario.lower()))
    palabras_material = Counter(nltk.word_tokenize(material.lower()))
    cobertura = sum(palabras_material[p] for p in palabras_temario if p in palabras_material) / len(palabras_temario)
    return cobertura

In [11]:
def evaluar_material_didactico(temario, material):
    """
    Evalúa la calidad de un material didáctico generado.
    """
    material_texto = concatenar_material(material)
    relevancia = calcular_relevancia(temario, material_texto)
    legibilidad = calcular_legibilidad(material_texto)
    terminologia = analizar_terminologia(temario, material_texto)
    return relevancia, legibilidad, terminologia

In [12]:
def select_file():
    """
    Opens a file dialog to select PDF, DOCX, or TXT files.
    Returns the selected file path or None if canceled.
    """

    # Create and show the root window
    root = tk.Tk()
    root.attributes('-topmost', True)  # Make dialog appear on top
    
    # Define allowed file types
    filetypes = (
        ('PDF files', '*.pdf'),
        ('Word documents', '*.docx'),
        ('Text files', '*.txt')
    )

    try:
        # Open file dialog
        file_path = filedialog.askopenfilename(
            parent=root,
            title='Select a file',
            filetypes=filetypes
        )
        
        # Destroy the root window after selection
        root.destroy()
        
        if file_path:
            return file_path
        return None

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        root.destroy()
        return None

In [ ]:
def generar_material_didactico(nombre_archivo):
    """
    Genera materiales didácticos basados en un temario y evalúa su calidad.
    """
    archivo = select_file()
    temario = extract_text(archivo)
    materiales = generate_study_materials(temario)
    evaluaciones = evaluar_material_didactico(temario, materiales)
    generate_pdf(materiales, nombre_archivo)
    print(f"\n=== Evaluación del Material Didáctico según el temario otorgado ===")
    print(f"Relevancia: {evaluaciones[0]:.2f}")
    print(f"Legibilidad: {evaluaciones[1]:.2f}")
    print(f"Terminología: {evaluaciones[2]:.2f}")

In [24]:
generar_material_didactico(input("Nombre del archivo que vas a generar: "))

✅ PDF generado correctamente: material_fp.pdf

=== Evaluación del Material Didáctico ===
Relevancia: 0.47
Legibilidad: 10.90
Terminología: 21.77
